<a href="https://colab.research.google.com/github/puzis/llmnet/blob/main/experiments/gpt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
dataset = load_dataset("ag_news")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# Tokenize the dataset
num_examples = 10
tokenized_dataset = dataset['train'].select([i for i in range(num_examples)]).map(
    lambda example: tokenizer(example['text'], padding='longest', truncation=True, max_length=256),
    batched=True
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
id = torch.tensor(tokenized_dataset['input_ids'][2])

In [ ]:
id.shape

torch.Size([98])

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import torch
import torch.nn as nn

class GPT2FeatureExtractor(nn.Module):
    def __init__(self, gpt2_model):
        super(GPT2FeatureExtractor, self).__init__()
        self.gpt2_model = gpt2_model
        self.layer_features = {'ln_1': [], 'ln_2': [], 'attn': [], 'mlp': []}
        self.nlf_features = []  # Updated to store as a list
        self.embedding_features = None  # Updated to store as a tensor

        # Register hooks for each module
        for i, layer in enumerate(self.gpt2_model.transformer.h):
            layer_name = f"layer_{i + 1}"
            self.layer_features[layer_name] = {
                'ln_1': None,
                'ln_2': None,
                'attn': None,
                'mlp': None
            }

            layer.ln_1.register_forward_hook(self.ln_1_hook(layer_name))
            layer.ln_2.register_forward_hook(self.ln_2_hook(layer_name))
            layer.attn.register_forward_hook(self.attn_hook(layer_name))
            layer.mlp.register_forward_hook(self.mlp_hook(layer_name))

        # Register hook for ln_f after the last layer
        self.gpt2_model.transformer.ln_f.register_forward_hook(self.nlf_hook)

        # Register hook for the embedding layer
        self.gpt2_model.transformer.wte.register_forward_hook(self.embedding_hook)

    def ln_1_hook(self, layer_name):
        def hook(module, input, output):
            self.layer_features[layer_name]['ln_1'] = output
        return hook

    def ln_2_hook(self, layer_name):
        def hook(module, input, output):
            self.layer_features[layer_name]['ln_2'] = output
        return hook

    def attn_hook(self, layer_name):
        def hook(module, input, output):
            self.layer_features[layer_name]['attn'] = output
        return hook

    def mlp_hook(self, layer_name):
        def hook(module, input, output):
            self.layer_features[layer_name]['mlp'] = output
        return hook

    def nlf_hook(self, module, input, output):
        # Store the hidden features of ln_f after the last layer
        self.nlf_features.append(output)

    def embedding_hook(self, module, input, output):
        # Store the hidden features of the embedding layer
        self.embedding_features = output

    def forward(self, input_ids, attention_mask=None):
        # Reset hidden features
        for layer_name in self.layer_features:
            for sub_module in self.layer_features[layer_name]:
                self.layer_features[layer_name][sub_module] = None

        self.nlf_features = []  # Reset ln_f features
        self.embedding_features = None  # Reset embedding features

        # Forward pass through the GPT-2 model
        outputs = self.gpt2_model(input_ids, attention_mask=attention_mask)

        # Return the stored hidden features
        return {
            'layers': self.layer_features,
            'nlf': self.nlf_features,
            'embedding': self.embedding_features
        }

# Instantiate the feature extractor
feature_extractor = GPT2FeatureExtractor(model)
# Example input

input_ids = torch.tensor(tokenized_dataset['input_ids'][2]).unsqueeze(0)
#input_ids = torch.tensor([1, 2, 3, 4, 5]).unsqueeze(0)

# Get hidden features
hidden_features = feature_extractor(input_ids)

IndexError: index out of range in self

In [ ]:
torch.tensor([1, 2, 3, 4, 5]).shape

torch.Size([5])

In [ ]:
list(hidden_features.keys())

['layers', 'nlf', 'embedding']

In [ ]:
hidden_features['embedding'].shape
for i in range(1, 13):
  hidden_features['layers'][f'layer_{i}']['ln_1'].shape
  hidden_features['layers'][f'layer_{i}']['attn'][0].shape
  hidden_features['layers'][f'layer_{i}']['attn'][1][0].shape
  hidden_features['layers'][f'layer_{i}']['attn'][1][1].shape
  hidden_features['layers'][f'layer_{i}']['ln_2'].shape
  hidden_features['layers'][f'layer_{i}']['mlp'].shape
hidden_features['nlf'][0].shape

torch.Size([1, 4, 768])

In [ ]:
print("Shape of embedding layer:", hidden_features['embedding'].shape)

for i in range(1, 13):
    print(f"\nLayer {i}:")
    print("  Shape of ln_1:", hidden_features['layers'][f'layer_{i}']['ln_1'].shape)
    print("  Shape of attn[0]:", hidden_features['layers'][f'layer_{i}']['attn'][0].shape)
    print("  Shape of attn[1][0]:", hidden_features['layers'][f'layer_{i}']['attn'][1][0].shape)
    print("  Shape of attn[1][1]:", hidden_features['layers'][f'layer_{i}']['attn'][1][1].shape)
    print("  Shape of ln_2:", hidden_features['layers'][f'layer_{i}']['ln_2'].shape)
    print("  Shape of mlp:", hidden_features['layers'][f'layer_{i}']['mlp'].shape)

print("\nShape of nlf:", hidden_features['nlf'][0].shape)

Shape of embedding layer: torch.Size([1, 6, 768])

Layer 1:
  Shape of ln_1: torch.Size([1, 6, 768])
  Shape of attn[0]: torch.Size([1, 6, 768])
  Shape of attn[1][0]: torch.Size([1, 12, 6, 64])
  Shape of attn[1][1]: torch.Size([1, 12, 6, 64])
  Shape of ln_2: torch.Size([1, 6, 768])
  Shape of mlp: torch.Size([1, 6, 768])

Layer 2:
  Shape of ln_1: torch.Size([1, 6, 768])
  Shape of attn[0]: torch.Size([1, 6, 768])
  Shape of attn[1][0]: torch.Size([1, 12, 6, 64])
  Shape of attn[1][1]: torch.Size([1, 12, 6, 64])
  Shape of ln_2: torch.Size([1, 6, 768])
  Shape of mlp: torch.Size([1, 6, 768])

Layer 3:
  Shape of ln_1: torch.Size([1, 6, 768])
  Shape of attn[0]: torch.Size([1, 6, 768])
  Shape of attn[1][0]: torch.Size([1, 12, 6, 64])
  Shape of attn[1][1]: torch.Size([1, 12, 6, 64])
  Shape of ln_2: torch.Size([1, 6, 768])
  Shape of mlp: torch.Size([1, 6, 768])

Layer 4:
  Shape of ln_1: torch.Size([1, 6, 768])
  Shape of attn[0]: torch.Size([1, 6, 768])
  Shape of attn[1][0]: torc

In [ ]:
import torch

def concatenate_tensors(features):
    concatenated_tensors = []

    # Append the embedding layer
    concatenated_tensors.append(features['embedding'])

    # Iterate through the layers
    for i in range(1, 13):
        ln_1 = features['layers'][f'layer_{i}']['ln_1']
        attn_0 = features['layers'][f'layer_{i}']['attn'][0]
        attn_1_0 = features['layers'][f'layer_{i}']['attn'][1][0].reshape(1, 4, 768)
        attn_1_1 = features['layers'][f'layer_{i}']['attn'][1][1].reshape(1, 4, 768)
        ln_2 = features['layers'][f'layer_{i}']['ln_2']
        mlp = features['layers'][f'layer_{i}']['mlp']

        # Append tensors to the list
        concatenated_tensors.extend([ln_1, attn_0, attn_1_0, attn_1_1, ln_2, mlp])

    # Append the nlf tensor
    concatenated_tensors.append(features['nlf'][0])

    # Concatenate tensors along a new dimension (axis)
    result_tensor = torch.cat(concatenated_tensors, dim=0)

    return result_tensor

In [ ]:
result = concatenate_tensors(hidden_features)
print("Concatenated Result Shape:", result.shape)

RuntimeError: shape '[1, 4, 768]' is invalid for input of size 4608